<a href="https://colab.research.google.com/github/sunilganta84/Ecommerce-Sales-ML/blob/main/GANNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")

# Error handling with try-catch
try:
    # Load the conv_ai_3 dataset
    ds = tfds.load('huggingface:conv_ai_3/conv_ai_3', split='train')

    # Preprocess the text data
    tokenizer = tfds.deprecated.text.Tokenizer()
    vocabulary_set = set()

    for example in ds:
        text = example['question'].numpy().decode('utf-8')  # Access the question text
        tokens = tokenizer.tokenize(text)
        vocabulary_set.update(tokens)

    encoder = tfds.deprecated.text.TokenTextEncoder(list(vocabulary_set))  # Convert vocabulary_set to list

    # Define the Generator model
    class Generator(tf.keras.Model):
        def __init__(self, vocab_size, embedding_dim, rnn_units):
            super(Generator, self).__init__()
            self.embedding = Embedding(vocab_size, embedding_dim)
            self.lstm = LSTM(rnn_units, return_sequences=True, return_state=True)
            self.dense = Dense(vocab_size, activation='softmax')

        def call(self, x, hidden):
            x = self.embedding(x)
            x, state_h, state_c = self.lstm(x, initial_state=hidden)
            x = self.dense(x)
            return x[:, -1, :], [state_h, state_c]

    # Define the Discriminator model
    class Discriminator(tf.keras.Model):
        def __init__(self):
            super(Discriminator, self).__init__()
            self.dense = Dense(64, activation='relu')

        def call(self, x):
            x = tf.reshape(x, [-1, x.shape[1], 1])
            x = self.dense(x)
            return x

    # Define the loss function and optimizer
    loss_object = BinaryCrossentropy(from_logits=True)
    generator_optimizer = Adam(learning_rate=0.001)
    discriminator_optimizer = Adam(learning_rate=0.001)

    # Define the loss function for the Generator
    def generator_loss(fake_output):
        return loss_object(tf.ones_like(fake_output), fake_output)

    # Define the loss function for the Discriminator
    def discriminator_loss(real_output, fake_output):
        real_loss = loss_object(tf.ones_like(real_output), real_output)
        fake_loss = loss_object(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    # Define the training step
    @tf.function
    def train_step(real_text):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            initial_state = [tf.zeros((1, rnn_units)), tf.zeros((1, rnn_units))]

            generated_text, _ = generator(real_text, initial_state)
            real_output = discriminator(real_text)
            fake_output = discriminator(generated_text)

            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        return gen_loss, disc_loss

    # Define hyperparameters
    embedding_dim = 256
    rnn_units = 1024
    vocab_size = len(vocabulary_set)

    # Initialize the Generator and Discriminator models
    generator = Generator(vocab_size, embedding_dim, rnn_units)
    discriminator = Discriminator()

    # Train the GAN model
    EPOCHS = 10
    for epoch in range(EPOCHS):
        for example in ds:
            text = example['question'].numpy().decode('utf-8')  # Access the question text
            tokens = tokenizer.tokenize(text)
            filtered_tokens = [token for token in tokens if token in vocabulary_set]
            if filtered_tokens:
                print("Filtered Tokens:", filtered_tokens)  # Debugging statement
                encoded_text = encoder.encode(' '.join(filtered_tokens))  # Join tokens into a single string
                real_text = tf.expand_dims(encoded_text, 0)
                real_text = pad_sequences(real_text, padding='post')  # Pad the input sequences

                try:
                    gen_loss, disc_loss = train_step(real_text)
                except tf.errors.InvalidArgumentError as error:
                    print("InvalidArgumentError:", error)
                    # Handle error gracefully, e.g., by logging and continuing training or by terminating
                    # showErrorMessage(error)

        print(f'Epoch {epoch+1}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}')

except Exception as error:
    # log error
    print(error)
    # handle error
    showErrorMessage(error)


Generating train split:   0%|          | 0/9176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2313 [00:00<?, ? examples/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/conv_ai_3/conv_ai_3/1.0.0.incompleteZ51DLK/conv_ai_3-train.tfrecord*...:  …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/conv_ai_3/conv_ai_3/1.0.0.incompleteZ51DLK/conv_ai_3-validation.tfrecord*.…

Dataset conv_ai_3 downloaded and prepared to /root/tensorflow_datasets/conv_ai_3/conv_ai_3/1.0.0. Subsequent calls will reuse this data.
Filtered Tokens: ['are', 'you', 'interested', 'in', 'seeing', 'barack', 'obamas', 'family']
Filtered Tokens: ['would', 'you', 'like', 'to', 'know', 'barack', 'obamas', 'geneology']
Filtered Tokens: ['would', 'you', 'like', 'to', 'know', 'about', 'obamas', 'ancestors']
Filtered Tokens: ['would', 'you', 'like', 'to', 'know', 'who', 'is', 'currently', 'alive', 'from', 'president', 'obamas', 'family', 'tree']
Filtered Tokens: ['are', 'you', 'looking', 'for', 'biological', 'information', 'on', 'his', 'family']
Filtered Tokens: ['are', 'you', 'referring', 'to', 'the', 'time', 'magazine', 'essay']
Filtered Tokens: ['do', 'you', 'want', 'to', 'know', 'more', 'about', 'obamas', 'parents']
Filtered Tokens: ['how', 'many', 'generations', 'would', 'you', 'like', 'me', 'to', 'list', 'from', 'president', 'obamas', 'family', 'tree']
Filtered Tokens: ['what', 'would'

Filtered Tokens: ['would', 'you', 'like', 'to', 'hear', 'about', 'president', 'obamas', 'family', 'tree']


Streaming output truncated to the last 5000 lines.
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 738, in _run_callback

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 825, in inner

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 786, in run

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 361, in process_one

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 234, in wrapper

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 261, in dispatch_shell

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 234, in wra